## Semaine 5 : Réseaux de neurones

Après avoir réaliser un algorithme qui récupère à partir d'un jeu de données les caractéristiques de chaque trajectoires (Heure de départ, heure d'arrivé, vitesse, durée...). Nous allons exploiter ces données dans un réseau de neurones pour pourvoir à partir de ses caractéristiques prédire la trajectoire au futur.

#### Plan : 

##### 1. Qu'est-ce qu'un réseau de neurones?

##### 2. Comprendre le processus

##### 3. Implémentation d'un réseau de neurones sur nos données

##### 4. Conclusion

.

    1. Qu'est-ce qu'un réseau de neurones?

Avec environ 100 milliards de neurones, le cerveau humain traite les données à des vitesses aussi rapides que 268 mph! En substance, un réseau de neurones est une collection de neurones connectés par des synapses . Cette collection est organisée en trois couches principales: la couche d'entrée, la couche cachée et la couche de sortie. Vous pouvez avoir de nombreuses couches cachées, c'est là que le terme d' apprentissage profond entre en jeu. Dans un réseau de neurones artificiels, il existe plusieurs entrées, appelées caractéristiques , qui produisent une sortie unique, appelée étiquette .

Les cercles représentent les neurones tandis que les lignes représentent les synapses. Le rôle d'une synapse est de multiplier les intrants et les poids . Vous pouvez considérer les poids comme la «force» de la connexion entre les neurones. Les poids définissent principalement la sortie d'un réseau de neurones. Cependant, ils sont très flexibles. Après, une fonction d'activation est appliquée pour renvoyer une sortie.

    2. Comprendre le processus

Voici un bref aperçu de la façon dont fonctionne un réseau de neurones feedforward simple:

    * Prend des entrées en tant que matrice (tableau de nombres 2D)

    * Multiplie l'entrée par un ensemble de poids (effectue un produit scalaire alias multiplication de la matrice)

    * Applique une fonction d'activation

    * Renvoie une sortie

    * L'erreur est calculée en prenant la différence par rapport à la sortie désirée des données et à la sortie prévue. Cela crée notre descente de gradient, que nous pouvons utiliser pour modifier les poids

    * Les poids sont ensuite légèrement modifiés en fonction de l'erreur.

Pour s'entraîner, ce processus est répété 1 000 fois et plus. Plus les données sont formées, plus nos résultats seront précis.

À la base, les réseaux de neurones sont simples. Ils effectuent simplement un produit scalaire avec l'entrée et pondère et appliquent une fonction d'activation. Lorsque les poids sont ajustés via la fonction de gradient de perte, le réseau s'adapte aux changements pour produire des sorties plus précises.

Notre réseau de neurones modélisera une seule couche cachée avec trois entrées et deux sorties.

Dans le réseau, nous prédirons latitude et la longitude de notre notre position à partir des entrées (heure, vitesse, acceleration).

#### Propagation vers l'avant

Tout d'abord, importons les biblithèques nécessaires et normalisons nos données sous la forme de tableaux numériques np.array.

In [2]:
import numpy as np
import extraire
import NN7
import filters
import gmplot
from IPython.display import IFrame
import matplotlib.pyplot as plt

In [3]:
android_df = extraire.importJson("data/android_small.json", True)

In [4]:
day_df = extraire.selectDate("25-11-2017", android_df) #Nous avons choisi une journée au hasard

In [5]:
import staypoint as st

In [6]:
day_df = filters.meanFilter(day_df, 10)

In [7]:
stay_point_df = st.findStayPoints(day_df, 3, 50, 5)

In [16]:
X = np.array(NN7.approxiamtion(stay_point_df,True),dtype=float)
y = np.array(NN7.approxiamtion2(stay_point_df,True),dtype=float)

In [17]:
print("Entrée : Vitesse, Acceleration, Heure \n",X)
print("Sortie : Latitude, Longitude \n",y)


Entrée : Vitesse, Acceleration, Heure 
 [[4.98547451e+01 7.06759497e+00 8.01277097e+04]
 [3.53106837e+00 4.11247016e-01 7.19004167e+04]
 [2.73002814e+00 1.69783291e-01 5.84640588e+04]
 [5.25026850e+00 1.45762599e+01 5.66042353e+04]
 [1.31694281e+01 2.31525013e+00 4.16038788e+04]
 [4.04491639e+01 1.21778693e+01 3.63105439e+04]
 [6.77823281e+00 6.51463455e-01 3.13218814e+04]
 [3.63484533e+00 1.91717538e-01 1.26623077e+03]]
Sortie : Latitude, Longitude 
 [[45.89934027  4.7252978 ]
 [45.75383879  4.82576098]
 [45.76194822  4.83011616]
 [45.76160869  4.82974817]
 [45.75872127  4.88351209]
 [45.77123589  4.87214872]
 [45.77583265  4.81230044]
 [45.7599635   4.83422598]]


Ensuite, définissons un python classet écrivons une initfonction dans laquelle nous spécifierons nos paramètres tels que les couches input, hidden et output.

In [18]:
class Neural_Network(object):
    
    def __init__(self):
       
        self.inputSize = 3
        self.outputSize = 2
        self.hiddenSize = 4

Il est temps pour notre premier calcul. Rappelez-vous que nos synapses effectuent un produit scalaire , ou une multiplication matricielle de l'entrée et du poids. Notez que les poids sont générés aléatoirement et entre 0 et 1.

Dans l'ensemble de données, nos données d'entrée X sont une matrice len(tableau)x3. Nos données de sortie y, est une matrice len(tableau)x2. Chaque élément de la matrice X doit être multiplié par un poids correspondant, puis ajouté avec tous les autres résultats pour chaque neurone dans la couche cachée.

Pour obtenir la valeur finale de la couche cachée, nous devons appliquer la fonction d'activation . Le rôle d'une fonction d'activation est d'introduire la non-linéarité. Un avantage de ceci est que la sortie est mappée à partir d'une plage de 0 et 1, ce qui rend plus facile de modifier les poids dans le futur.

Il y a beaucoup de fonctions d'activation là-bas. Dans ce cas, nous allons nous en tenir à l'un des plus populaires - la fonction sigmoïde.

Maintenant, nous devons utiliser à nouveau la multiplication matricielle, avec un autre ensemble de poids aléatoires, pour calculer la valeur de la couche de sortie.

Enfin, pour normaliser la sortie, nous appliquons à nouveau la fonction d'activation.

### Rétropropagation
    1. "L'apprentissage" de notre réseau
Puisque nous avons un ensemble aléatoire de poids, nous devons les modifier pour rendre nos entrées égales aux sorties correspondantes de notre ensemble de données. Ceci est fait par une méthode appelée backpropagation.

La rétropropagation fonctionne en utilisant une fonction de perte pour calculer la distance du réseau par rapport à la sortie cible.

    2. Erreur de calcul

Une façon de représenter la fonction de perte est d'utiliser la fonction de perte au carré moyen :


Dans cette fonction, oest notre sortie prédite, et yest notre sortie réelle. Maintenant que nous avons la fonction de perte, notre objectif est de le rapprocher autant que possible de 0. Cela signifie que nous n'aurons presque aucune perte. Comme nous formons notre réseau, tout ce que nous faisons, c'est minimiser la perte.

Pour déterminer quelle direction modifier nos poids, nous devons trouver le taux de variation de notre perte par rapport à nos poids. En d'autres termes, nous devons utiliser la dérivée de la fonction de perte pour comprendre comment les poids affectent l'entrée.

Dans ce cas, nous utiliserons une dérivée partielle pour nous permettre de prendre en compte une autre variable.

Cette méthode est appelée descente en gradient . En sachant de quelle manière modifier nos poids, nos résultats ne peuvent qu'être plus précis.

Voici comment nous calculerons le changement incrémental de nos poids:

    * Trouver la marge d'erreur de la couche de sortie (o) en prenant la différence entre la sortie prédite et la sortie réelle (y)

    * Appliquez la dérivée de notre fonction d'activation sigmoïde à l'erreur de la couche de sortie. Nous appelons ce résultat la somme de sortie delta .

    * Utilisez la somme de sortie delta de l'erreur de couche de sortie pour déterminer à quel point notre couche z 2 (cachée) a contribué à l'erreur de sortie en effectuant un produit scalaire avec notre deuxième matrice de poids. Nous pouvons appeler cela l' erreur z 2 .

    * Calculer la somme de sortie delta pour la couche z 2 en appliquant la dérivée de notre fonction d'activation sigmoïde (tout comme l'étape 2).

    * Ajustez les poids pour la première couche en effectuant un produit scalaire de la couche d'entrée avec la somme de sortie delta cachée (z 2 ) . Pour la deuxième couche, exécutez un produit scalaire de la couche cachée (z 2 ) et de la sortie de sortie delta (o) .

Le calcul de la somme de sortie delta et ensuite l'application de la dérivée de la fonction sigmoïde sont très importants pour la rétropropagation. La dérivée du sigmoïde, également appelée sigmoïde prime , nous donnera le taux de variation, ou pente, de la fonction d'activation à la somme de sortie.

Ensuite, nous voulons créer notre backwardfonction de propagation qui fait tout ce qui est spécifié dans les quatre étapes ci-dessus:

Nous pouvons maintenant définir notre sortie en initiant la propagation vers l'avant et intiate la fonction en arrière en l'appelant dans la trainfonction:

Pour exécuter le réseau, tout ce que nous avons à faire est d'exécuter la trainfonction. Bien sûr, nous voudrons faire cela plusieurs fois, ou peut-être des milliers de fois. Donc, nous allons utiliser une forboucle.

### * Prédictions

Pour prédire notre position pour l'entrée (data disponible + une fluctuation de 0.01), nous aurons besoin de créer un nouveau tableau pour stocker ces données xPredicted.

Ensuite, nous allons créer une nouvelle fonction qui imprime notre sortie prévue pour xPredicted. Tout ce que nous avons à faire est forward(xPredicted) de retourner une sortie.

    3. Implementation du code 

In [19]:
X = np.array(NN7.approxiamtion(stay_point_df,True),dtype=float)
y = np.array(NN7.approxiamtion2(stay_point_df,True),dtype=float)
xPredicted = np.array(NN7.approxiamtionprediction(stay_point_df,True), dtype=float)

# unités d'échelle
X = X/np.amax(X, axis=0) # maximum de tableau X
xPredicted = xPredicted/np.amax(xPredicted, axis=0) # maximum de xPredicted (nos données d'entrée pour la prédiction)
y = y/100 

class Neural_Network(object):
    
    def __init__(self):
        #parameters
        self.inputSize = 3
        self.outputSize = 2
        self.hiddenSize = 4

        #Poids
        self.W1 = np.random.randn(self.inputSize, self.hiddenSize) #  matrice de poids de l'entrée à la couche cachée
        self.W2 = np.random.randn(self.hiddenSize, self.outputSize) #  matrice de poids de caché à la couche de sortie

    def forward(self, X):
        # propagation directe à travers notre réseau
        self.z = np.dot(X, self.W1) # produit scalaire de X (entrée) et premier ensemble de poids
        self.z2 = self.sigmoid(self.z) # Fonction d'activation
        self.z3 = np.dot(self.z2, self.W2) # produit scalaire de couche cachée (z2) et deuxième ensemble de poids
        o = self.sigmoid(self.z3) # fonction d'activation
        return o

    def sigmoid(self, s):
        # fonction d'activaton
        return 1/(1+np.exp(-s))

    def sigmoidPrime(self, s):
        # dérivé du sigmoïde
        return s * (1 - s)

    def backward(self, X, y, o):
        # propagation vers l'arrière à travers le réseau
        self.o_error = y - o # Erreur de l'entrée
        self.o_delta = self.o_error*self.sigmoidPrime(o) # Aplliquer la dérivé du sigmoide sur l'erreur

        self.z2_error = self.o_delta.dot(self.W2.T) # Erreur z2: combien nos poids de couche cachés ont contribué à l'erreur de sortie
        self.z2_delta = self.z2_error*self.sigmoidPrime(self.z2) # application de la dérivée de l'erreur sigmoïde à z2

        self.W1 += X.T.dot(self.z2_delta) # ajuster le premier ensemble (entrée -> caché) poids
        self.W2 += self.z2.T.dot(self.o_delta) # ajuster le deuxième ensemble (caché -> sortie) poids

    def train(self, X, y):
        o = self.forward(X)
        self.backward(X, y, o)

    def saveWeights(self):
        np.savetxt("w1.txt", self.W1, fmt="%s")
        np.savetxt("w2.txt", self.W2, fmt="%s")

    def predict(self):
        print ("Données prédites basées sur des poids formés: ")
        print ("Input (ajusté): \n" + str(xPredicted))
        print ("Output: \n" + str(self.forward(xPredicted)))

NN = Neural_Network()
for i in range(1000): # Entrainement de réseua : 1000 fois
    print ("# " + str(i) + "\n")
    print ("Input (ajusté): \n" + str(X))
    print ("Ouput actual: \n" + str(y))
    print ("Ouput par prédiction: \n" + str(NN.forward(X)))
    a = NN.forward(X)
    print ("Erreur: \n" + str(np.mean(np.square(y - a)))) # Perte moyenne au carré
    print ("\n")
    b = NN.train(X, y)
    b
    
NN.saveWeights()
NN.predict()

# 0

Input (ajusté): 
[[1.         0.48487026 1.        ]
 [0.07082713 0.02821348 0.89732275]
 [0.05475964 0.01164793 0.72963597]
 [0.10531131 1.         0.70642522]
 [0.26415596 0.15883705 0.51921962]
 [0.8113403  0.83545912 0.45315839]
 [0.13595963 0.04469346 0.3908995 ]
 [0.07290871 0.01315273 0.01580266]]
Ouput actual: 
[[0.4589934  0.04725298]
 [0.45753839 0.04825761]
 [0.45761948 0.04830116]
 [0.45761609 0.04829748]
 [0.45758721 0.04883512]
 [0.45771236 0.04872149]
 [0.45775833 0.048123  ]
 [0.45759963 0.04834226]]
Ouput par prédiction: 
[[0.41141545 0.17097449]
 [0.33961525 0.15936483]
 [0.32851248 0.14994876]
 [0.36528506 0.28043072]
 [0.33405888 0.1471891 ]
 [0.38878662 0.20865995]
 [0.31359722 0.13335843]
 [0.28712285 0.11604975]]
Erreur: 
0.015636939046273484


# 1

Input (ajusté): 
[[1.         0.48487026 1.        ]
 [0.07082713 0.02821348 0.89732275]
 [0.05475964 0.01164793 0.72963597]
 [0.10531131 1.         0.70642522]
 [0.26415596 0.15883705 0.51921962]
 [0.8113403  0.

# 77

Input (ajusté): 
[[1.         0.48487026 1.        ]
 [0.07082713 0.02821348 0.89732275]
 [0.05475964 0.01164793 0.72963597]
 [0.10531131 1.         0.70642522]
 [0.26415596 0.15883705 0.51921962]
 [0.8113403  0.83545912 0.45315839]
 [0.13595963 0.04469346 0.3908995 ]
 [0.07290871 0.01315273 0.01580266]]
Ouput actual: 
[[0.4589934  0.04725298]
 [0.45753839 0.04825761]
 [0.45761948 0.04830116]
 [0.45761609 0.04829748]
 [0.45758721 0.04883512]
 [0.45771236 0.04872149]
 [0.45775833 0.048123  ]
 [0.45759963 0.04834226]]
Ouput par prédiction: 
[[0.47568718 0.0351414 ]
 [0.46609915 0.04406349]
 [0.46138146 0.0434446 ]
 [0.44142176 0.09354518]
 [0.45734368 0.04278317]
 [0.45511811 0.05594716]
 [0.45269402 0.04160206]
 [0.44198746 0.04154693]]
Erreur: 
0.00020733193121566962


# 78

Input (ajusté): 
[[1.         0.48487026 1.        ]
 [0.07082713 0.02821348 0.89732275]
 [0.05475964 0.01164793 0.72963597]
 [0.10531131 1.         0.70642522]
 [0.26415596 0.15883705 0.51921962]
 [0.8113403

Erreur: 
0.00010086912826083462


# 161

Input (ajusté): 
[[1.         0.48487026 1.        ]
 [0.07082713 0.02821348 0.89732275]
 [0.05475964 0.01164793 0.72963597]
 [0.10531131 1.         0.70642522]
 [0.26415596 0.15883705 0.51921962]
 [0.8113403  0.83545912 0.45315839]
 [0.13595963 0.04469346 0.3908995 ]
 [0.07290871 0.01315273 0.01580266]]
Ouput actual: 
[[0.4589934  0.04725298]
 [0.45753839 0.04825761]
 [0.45761948 0.04830116]
 [0.45761609 0.04829748]
 [0.45758721 0.04883512]
 [0.45771236 0.04872149]
 [0.45775833 0.048123  ]
 [0.45759963 0.04834226]]
Ouput par prédiction: 
[[0.46199876 0.03231109]
 [0.46228475 0.04185776]
 [0.46029113 0.04157799]
 [0.45081585 0.0798273 ]
 [0.45817303 0.04072418]
 [0.45709593 0.04951812]
 [0.45624268 0.04026452]
 [0.45211043 0.04089467]]
Erreur: 
0.00010033037836644162


# 162

Input (ajusté): 
[[1.         0.48487026 1.        ]
 [0.07082713 0.02821348 0.89732275]
 [0.05475964 0.01164793 0.72963597]
 [0.10531131 1.         0.70642522]
 [0.2641559

 [0.07290871 0.01315273 0.01580266]]
Ouput actual: 
[[0.4589934  0.04725298]
 [0.45753839 0.04825761]
 [0.45761948 0.04830116]
 [0.45761609 0.04829748]
 [0.45758721 0.04883512]
 [0.45771236 0.04872149]
 [0.45775833 0.048123  ]
 [0.45759963 0.04834226]]
Ouput par prédiction: 
[[0.4590734  0.03274413]
 [0.46072523 0.04251461]
 [0.45947197 0.04231092]
 [0.45330251 0.07606845]
 [0.45826729 0.04133244]
 [0.45854744 0.04857795]
 [0.45685672 0.04109513]
 [0.45448932 0.04186526]]
Erreur: 
7.763121666282271e-05


# 218

Input (ajusté): 
[[1.         0.48487026 1.        ]
 [0.07082713 0.02821348 0.89732275]
 [0.05475964 0.01164793 0.72963597]
 [0.10531131 1.         0.70642522]
 [0.26415596 0.15883705 0.51921962]
 [0.8113403  0.83545912 0.45315839]
 [0.13595963 0.04469346 0.3908995 ]
 [0.07290871 0.01315273 0.01580266]]
Ouput actual: 
[[0.4589934  0.04725298]
 [0.45753839 0.04825761]
 [0.45761948 0.04830116]
 [0.45761609 0.04829748]
 [0.45758721 0.04883512]
 [0.45771236 0.04872149]
 [0.45775833

6.174464715872259e-05


# 279

Input (ajusté): 
[[1.         0.48487026 1.        ]
 [0.07082713 0.02821348 0.89732275]
 [0.05475964 0.01164793 0.72963597]
 [0.10531131 1.         0.70642522]
 [0.26415596 0.15883705 0.51921962]
 [0.8113403  0.83545912 0.45315839]
 [0.13595963 0.04469346 0.3908995 ]
 [0.07290871 0.01315273 0.01580266]]
Ouput actual: 
[[0.4589934  0.04725298]
 [0.45753839 0.04825761]
 [0.45761948 0.04830116]
 [0.45761609 0.04829748]
 [0.45758721 0.04883512]
 [0.45771236 0.04872149]
 [0.45775833 0.048123  ]
 [0.45759963 0.04834226]]
Ouput par prédiction: 
[[0.45763118 0.03357773]
 [0.45978064 0.04350983]
 [0.45893914 0.04336339]
 [0.4544693  0.07316809]
 [0.45832098 0.04227028]
 [0.45951768 0.04818087]
 [0.45715662 0.04221813]
 [0.45576834 0.0430827 ]]
Erreur: 
6.152767751790057e-05


# 280

Input (ajusté): 
[[1.         0.48487026 1.        ]
 [0.07082713 0.02821348 0.89732275]
 [0.05475964 0.01164793 0.72963597]
 [0.10531131 1.         0.70642522]
 [0.26415596 0.1588370

...

 [0.45759963 0.04834226]]
Ouput par prédiction: 
[[0.45634114 0.03729022]
 [0.45897655 0.04690197]
 [0.45845141 0.04694823]
 [0.45530753 0.06378437]
 [0.45839297 0.0456584 ]
 [0.46080266 0.04750918]
 [0.45732018 0.04615068]
 [0.45672306 0.04726037]]
Erreur: 
2.4104641362917573e-05


# 606

Input (ajusté): 
[[1.         0.48487026 1.        ]
 [0.07082713 0.02821348 0.89732275]
 [0.05475964 0.01164793 0.72963597]
 [0.10531131 1.         0.70642522]
 [0.26415596 0.15883705 0.51921962]
 [0.8113403  0.83545912 0.45315839]
 [0.13595963 0.04469346 0.3908995 ]
 [0.07290871 0.01315273 0.01580266]]
Ouput actual: 
[[0.4589934  0.04725298]
 [0.45753839 0.04825761]
 [0.45761948 0.04830116]
 [0.45761609 0.04829748]
 [0.45758721 0.04883512]
 [0.45771236 0.04872149]
 [0.45775833 0.048123  ]
 [0.45759963 0.04834226]]
Ouput par prédiction: 
[[0.45634046 0.03729904]
 [0.45897671 0.0469076 ]
 [0.45845144 0.04695423]
 [0.45530806 0.06376381]
 [0.45839299 0.04566457]
 [0.4608037  0.04750812]
 [0.4573199  0

 [0.45667802 0.04780539]]
Erreur: 
1.9820346544146898e-05


# 692

Input (ajusté): 
[[1.         0.48487026 1.        ]
 [0.07082713 0.02821348 0.89732275]
 [0.05475964 0.01164793 0.72963597]
 [0.10531131 1.         0.70642522]
 [0.26415596 0.15883705 0.51921962]
 [0.8113403  0.83545912 0.45315839]
 [0.13595963 0.04469346 0.3908995 ]
 [0.07290871 0.01315273 0.01580266]]
Ouput actual: 
[[0.4589934  0.04725298]
 [0.45753839 0.04825761]
 [0.45761948 0.04830116]
 [0.45761609 0.04829748]
 [0.45758721 0.04883512]
 [0.45771236 0.04872149]
 [0.45775833 0.048123  ]
 [0.45759963 0.04834226]]
Ouput par prédiction: 
[[0.45629596 0.03802234]
 [0.45899565 0.04732406]
 [0.45845746 0.04739855]
 [0.45534956 0.06212827]
 [0.45839423 0.04613393]
 [0.46087533 0.04743781]
 [0.45729589 0.04668526]
 [0.45667747 0.04781082]]
Erreur: 
1.9777160630040712e-05


# 693

Input (ajusté): 
[[1.         0.48487026 1.        ]
 [0.07082713 0.02821348 0.89732275]
 [0.05475964 0.01164793 0.72963597]
 [0.10531131 1.      

# 782

Input (ajusté): 
[[1.         0.48487026 1.        ]
 [0.07082713 0.02821348 0.89732275]
 [0.05475964 0.01164793 0.72963597]
 [0.10531131 1.         0.70642522]
 [0.26415596 0.15883705 0.51921962]
 [0.8113403  0.83545912 0.45315839]
 [0.13595963 0.04469346 0.3908995 ]
 [0.07290871 0.01315273 0.01580266]]
Ouput actual: 
[[0.4589934  0.04725298]
 [0.45753839 0.04825761]
 [0.45761948 0.04830116]
 [0.45761609 0.04829748]
 [0.45758721 0.04883512]
 [0.45771236 0.04872149]
 [0.45775833 0.048123  ]
 [0.45759963 0.04834226]]
Ouput par prédiction: 
[[0.45626985 0.03871697]
 [0.45901728 0.04764371]
 [0.45846521 0.04773865]
 [0.45538772 0.06066383]
 [0.45839475 0.04651849]
 [0.46092338 0.04740592]
 [0.45727271 0.0471044 ]
 [0.45662876 0.04822879]]
Erreur: 
1.6378636402869044e-05


# 783

Input (ajusté): 
[[1.         0.48487026 1.        ]
 [0.07082713 0.02821348 0.89732275]
 [0.05475964 0.01164793 0.72963597]
 [0.10531131 1.         0.70642522]
 [0.26415596 0.15883705 0.51921962]
 [0.81134

Erreur: 
1.3929540881207693e-05


# 866

Input (ajusté): 
[[1.         0.48487026 1.        ]
 [0.07082713 0.02821348 0.89732275]
 [0.05475964 0.01164793 0.72963597]
 [0.10531131 1.         0.70642522]
 [0.26415596 0.15883705 0.51921962]
 [0.8113403  0.83545912 0.45315839]
 [0.13595963 0.04469346 0.3908995 ]
 [0.07290871 0.01315273 0.01580266]]
Ouput actual: 
[[0.4589934  0.04725298]
 [0.45753839 0.04825761]
 [0.45761948 0.04830116]
 [0.45761609 0.04829748]
 [0.45758721 0.04883512]
 [0.45771236 0.04872149]
 [0.45775833 0.048123  ]
 [0.45759963 0.04834226]]
Ouput par prédiction: 
[[0.45625804 0.03932014]
 [0.45903381 0.04786317]
 [0.45847054 0.04797042]
 [0.4554199  0.0594853 ]
 [0.4583951  0.04680334]
 [0.46095196 0.047409  ]
 [0.45725416 0.04740233]
 [0.45658842 0.04851246]]
Erreur: 
1.3903231013452773e-05


# 867

Input (ajusté): 
[[1.         0.48487026 1.        ]
 [0.07082713 0.02821348 0.89732275]
 [0.05475964 0.01164793 0.72963597]
 [0.10531131 1.         0.70642522]
 [0.2641559

 [0.07290871 0.01315273 0.01580266]]
Ouput actual: 
[[0.4589934  0.04725298]
 [0.45753839 0.04825761]
 [0.45761948 0.04830116]
 [0.45761609 0.04829748]
 [0.45758721 0.04883512]
 [0.45771236 0.04872149]
 [0.45775833 0.048123  ]
 [0.45759963 0.04834226]]
Ouput par prédiction: 
[[0.45625509 0.03990638]
 [0.45904596 0.04803093]
 [0.45847335 0.04814531]
 [0.45545048 0.05842294]
 [0.45839548 0.04704078]
 [0.46096975 0.04743886]
 [0.45723812 0.04763827]
 [0.45655335 0.04872315]]
Erreur: 
1.185231648781169e-05


# 954

Input (ajusté): 
[[1.         0.48487026 1.        ]
 [0.07082713 0.02821348 0.89732275]
 [0.05475964 0.01164793 0.72963597]
 [0.10531131 1.         0.70642522]
 [0.26415596 0.15883705 0.51921962]
 [0.8113403  0.83545912 0.45315839]
 [0.13595963 0.04469346 0.3908995 ]
 [0.07290871 0.01315273 0.01580266]]
Ouput actual: 
[[0.4589934  0.04725298]
 [0.45753839 0.04825761]
 [0.45761948 0.04830116]
 [0.45761609 0.04829748]
 [0.45758721 0.04883512]
 [0.45771236 0.04872149]
 [0.45775833

Nous remarquons que après 1000 exécutions de notre algorithme, l'erreur est de 1.0931 x 10^-5. Ce qui est très petit comme marge d'erreur.

    4. Conclusion

Un réseau de neurones à part entière capable d'apprendre et de s'adapter pour produire des résultats précis. On pensait à créer un notebook interactif qui permettra de voir en temps réel la prédiction retrouvée sur la carte géographique mais cela n'a pas abouti. Les calculs que nous avons faits, aussi complexes qu'ils semblaient être, ont tous joué un grand rôle dans notre modèle d'apprentissage. Si vous y réfléchissez, c'est impressionnant que votre ordinateur, un objet physique, ait réussi à apprendre par lui-même!